In [97]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

---
**Préparation du jeu de données**

In [98]:
df = pd.read_csv("results.csv")

In [99]:
df = df.loc[df["tournament"] == "African Cup of Nations"]

In [100]:
df.drop(columns=["tournament"], inplace=True)

In [101]:
df.head(5)

,date,home_team,away_team,home_score,away_score,city,country,neutral
4099,1957-02-10,Sudan,Egypt,1,2,Khartoum,Sudan,False
4100,1957-02-16,Egypt,Ethiopia,4,0,Khartoum,Sudan,True
4539,1959-05-22,Egypt,Ethiopia,4,0,Cairo,United Arab Republic,True
4542,1959-05-25,Ethiopia,Sudan,0,1,Cairo,United Arab Republic,True
4545,1959-05-29,Egypt,Sudan,2,1,Cairo,United Arab Republic,True


---
**Feature Engineering pour voir le nombre de victoires et de défaites par équipe**
On traitera le cas des matchs nuls plus tard

In [102]:
# Si l'équipe à domicile a gagné, la colonne home_win sera à 1 sinon away_win sera à 0
df['home_win'] = df['home_score'] > df['away_score']
df['away_win'] = df['away_score'] > df['home_score']

In [103]:
# Si l'équipe à domicile a gagné, la colonne home_win sera à 1 sinon away_win sera à 0
df['home_loss'] = df['home_score'] < df['away_score']
df['away_loss'] = df['away_score'] < df['home_score']

In [104]:
total_wins = pd.concat([df.groupby('home_team')['home_win'].sum(),
                        df.groupby('away_team')['away_win'].sum()], axis=1, sort=False).sum(axis=1)
total_wins

Algeria              28
Angola                4
Benin                 0
Botswana              0
Burkina Faso          9
Burundi               0
Cameroon             45
Cape Verde            2
Comoros               1
Congo                 7
DR Congo             20
Egypt                60
Equatorial Guinea     6
Ethiopia              7
Gabon                 7
Gambia                3
Ghana                54
Guinea               13
Guinea-Bissau         0
Ivory Coast          44
Kenya                 2
Liberia               1
Libya                 3
Madagascar            2
Malawi                2
Mali                 19
Mauritania            0
Mauritius             0
Morocco              27
Mozambique            0
Namibia               0
Niger                 0
Nigeria              53
Rwanda                1
Senegal              27
Sierra Leone          1
South Africa         16
Sudan                 7
Tanzania              0
Togo                  3
Tunisia              25
Uganda          

In [105]:
total_loss = pd.concat([df.groupby('home_team')['home_loss'].sum(),
                        df.groupby('away_team')['away_loss'].sum()], axis=1, sort=False).sum(axis=1)
total_loss

Algeria              27
Angola               10
Benin                 9
Botswana              3
Burkina Faso         23
Burundi               3
Cameroon             16
Cape Verde            3
Comoros               3
Congo                12
DR Congo             29
Egypt                27
Equatorial Guinea     5
Ethiopia             19
Gabon                 8
Gambia                1
Ghana                27
Guinea               17
Guinea-Bissau         6
Ivory Coast          28
Kenya                11
Liberia               2
Libya                 3
Madagascar            1
Malawi                6
Mali                 16
Mauritania            4
Mauritius             3
Morocco              19
Mozambique           10
Namibia               7
Niger                 5
Nigeria              22
Rwanda                1
Senegal              23
Sierra Leone          4
South Africa         14
Sudan                13
Tanzania              5
Togo                 14
Tunisia              26
Uganda          

---
**Tracer la courbe pour les 10 équipes avec le plus de victoires**

In [106]:
# Trier les équipes en fonction du nombre total de victoires
total_wins = total_wins.sort_values(ascending=False)

# Sélectionner les 10 premiers résultats
top_10_wins = total_wins.head(10)

# Définir des couleurs personnalisées pour chaque barre
colors = ['skyblue', 'lightcoral', 'lightgreen', 'gold', 'lightsalmon',
          'lightsteelblue', 'lightpink', 'lightseagreen', 'lightcyan', 'yellow']

# Tracer le graphique avec Plotly
fig = px.bar(top_10_wins, x=top_10_wins.index, y=top_10_wins.values, color=top_10_wins.index,
             color_discrete_sequence=colors, labels={'index': 'Équipe', 'y': 'Nombre total de victoires'})

# Ajouter le texte au-dessus de chaque barre
for i, value in enumerate(top_10_wins.values):
    fig.add_annotation(x=i, y=value, text=str(value), showarrow=False, font=dict(size=8))

# Personnaliser la disposition
fig.update_layout(
    title='Top 10 des équipes avec le plus grand nombre de victoires',
    xaxis_title='Équipe',
    yaxis_title='Nombre total de victoires',
    xaxis=dict(tickangle=-45, tickmode='array', tickvals=list(range(10)), ticktext=top_10_wins.index)
)

# Afficher le graphique
fig.show()

---
**Tracer la courbe pour les 10 équipes avec le plus de défaites**

In [107]:
# Trier les équipes en fonction du nombre total de victoires
total_loss = total_loss.sort_values(ascending=False)

# Sélectionner les 10 premiers résultats
top_10_loss = total_loss.head(10)

# Définir des couleurs personnalisées pour chaque barre
colors = ['skyblue', 'lightcoral', 'lightgreen', 'gold', 'lightsalmon',
          'lightsteelblue', 'lightpink', 'lightseagreen', 'lightcyan', 'yellow']

# Tracer le graphique avec Plotly
fig = px.bar(top_10_loss, x=top_10_loss.index, y=top_10_loss.values, color=top_10_loss.index,
             color_discrete_sequence=colors, labels={'index': 'Équipe', 'y': 'Nombre total de défaites'})

# Ajouter le texte au-dessus de chaque barre
for i, value in enumerate(top_10_loss.values):
    fig.add_annotation(x=i, y=value, text=str(value), showarrow=False, font=dict(size=8))

# Personnaliser la disposition
fig.update_layout(
    title='Top 10 des équipes avec le plus grand nombre de défaites',
    xaxis_title='Équipe',
    yaxis_title='Nombre total de défaites',
    xaxis=dict(tickangle=-45, tickmode='array', tickvals=list(range(10)), ticktext=top_10_loss.index)
)

# Afficher le graphique
fig.show()

**Nous allons refaire le même travail en se basant sur les pourcentages**

In [108]:
# Nombre total de matches par équipe
total_games = pd.concat([df['home_team'].value_counts(),
                         df['away_team'].value_counts()], axis=1, sort=False).sum(axis=1)

In [109]:
# Pourcentage de victoires
win_percentage = (total_wins / total_games) * 100

# Les 10 équipes avec les plus grands pourcentages de victoires
top_10_win_percentage = win_percentage.nlargest(10).reset_index()
top_10_win_percentage.columns = ['Équipe', 'Pourcentage de victoires']

top_10_win_percentage['Pourcentage de victoires'] = top_10_win_percentage['Pourcentage de victoires'].round(2)

# Tracer le graphique avec Plotly
fig = px.bar(top_10_win_percentage, x='Équipe', y='Pourcentage de victoires',
             title='Top 10 des équipes avec le plus haut pourcentage de victoires',
             labels={'Pourcentage de victoires': 'Pourcentage de victoires'},
             text='Pourcentage de victoires',
             height=500, color='Équipe', color_discrete_sequence=colors)

# Personnaliser l'apparence
fig.update_traces(marker_line_color='black', marker_line_width=1.5, textposition='outside')
fig.update_layout(xaxis=dict(tickangle=45, tickmode='array', tickvals=list(range(10)), ticktext=top_10_win_percentage['Équipe']))

# Afficher le graphique
fig.show()

In [110]:
# Pourcentage de défaites
loss_percentage = (total_loss / total_games) * 100

# Les 10 équipes avec les plus grands pourcentages de défaites
top_10_loss_percentage = loss_percentage.nlargest(10).reset_index()
top_10_loss_percentage.columns = ['Équipe', 'Pourcentage de défaites']

top_10_loss_percentage['Pourcentage de défaites'] = top_10_loss_percentage['Pourcentage de défaites'].round(2)

# Tracer le graphique avec Plotly
fig = px.bar(top_10_loss_percentage, x='Équipe', y='Pourcentage de défaites',
             title='Top 10 des équipes avec le plus haut pourcentage de défaites',
             labels={'Pourcentage de défaites': 'Pourcentage de défaites'},
             text='Pourcentage de défaites',
             height=500, color='Équipe', color_discrete_sequence=colors)

# Personnaliser l'apparence
fig.update_traces(marker_line_color='black', marker_line_width=1.5, textposition='outside')
fig.update_layout(xaxis=dict(tickangle=45, tickmode='array', tickvals=list(range(10)), ticktext=top_10_loss_percentage['Équipe']))

# Afficher le graphique
fig.show()

In [111]:
# Les équipes qui ont joué le plus de matchs dans la compétition
total_games = total_games.sort_values(ascending=False)
top_10_games = total_games.head(10)

top_10_games

Egypt          107
Ghana          102
Ivory Coast     99
Nigeria         96
Cameroon        91
Tunisia         79
Algeria         77
DR Congo        73
Morocco         70
Zambia          67
dtype: int64

In [112]:
type(top_10_games)

pandas.core.series.Series

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 741 entries, 4099 to 43562
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        741 non-null    object
 1   home_team   741 non-null    object
 2   away_team   741 non-null    object
 3   home_score  741 non-null    int64 
 4   away_score  741 non-null    int64 
 5   city        741 non-null    object
 6   country     741 non-null    object
 7   neutral     741 non-null    bool  
 8   home_win    741 non-null    bool  
 9   away_win    741 non-null    bool  
 10  home_loss   741 non-null    bool  
 11  away_loss   741 non-null    bool  
dtypes: bool(5), int64(2), object(5)
memory usage: 49.9+ KB


In [114]:
df["date"] = pd.to_datetime(df["date"])

In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 741 entries, 4099 to 43562
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        741 non-null    datetime64[ns]
 1   home_team   741 non-null    object        
 2   away_team   741 non-null    object        
 3   home_score  741 non-null    int64         
 4   away_score  741 non-null    int64         
 5   city        741 non-null    object        
 6   country     741 non-null    object        
 7   neutral     741 non-null    bool          
 8   home_win    741 non-null    bool          
 9   away_win    741 non-null    bool          
 10  home_loss   741 non-null    bool          
 11  away_loss   741 non-null    bool          
dtypes: bool(5), datetime64[ns](1), int64(2), object(4)
memory usage: 49.9+ KB


In [116]:
# Créer une nouvelle colonne avec l'année de la CAN
df["year"] = df["date"].dt.year

In [117]:
df["year"].unique()

array([1957, 1959, 1962, 1963, 1965, 1968, 1970, 1972, 1974, 1976, 1978,
       1980, 1982, 1984, 1986, 1988, 1990, 1992, 1994, 1996, 1998, 2000,
       2002, 2004, 2006, 2008, 2010, 2012, 2013, 2015, 2017, 2019, 2022],
      dtype=int64)

In [118]:
# Constitution d'un df avec les équipes et le nombre d'éditions auxquels ils ont participé 
unique_home_teams_and_years = df[["home_team", "year"]].drop_duplicates()
unique_home_teams_and_years.rename(columns={'home_team': 'team'}, inplace=True)

unique_away_teams_and_years = df[["away_team", "year"]].drop_duplicates()
unique_away_teams_and_years.rename(columns={'away_team': 'team'}, inplace=True)

unique_teams_and_years = (pd.concat([unique_home_teams_and_years, unique_away_teams_and_years])).drop_duplicates()

unique_teams_and_years

,team,year
4099,Sudan,1957
4100,Egypt,1957
4539,Egypt,1959
4542,Ethiopia,1959
5076,Ethiopia,1962
...,...,...
32293,Benin,2010
32295,Tunisia,2010
34229,Angola,2012
34235,Tunisia,2012


In [119]:
dict_team_nb_editions = unique_teams_and_years["team"].value_counts().to_dict()
dict_team_nb_editions

{'Egypt': 25,
 'Ivory Coast': 24,
 'Ghana': 23,
 'Tunisia': 20,
 'Cameroon': 20,
 'Nigeria': 19,
 'DR Congo': 19,
 'Algeria': 19,
 'Morocco': 18,
 'Zambia': 17,
 'Senegal': 16,
 'Guinea': 13,
 'Burkina Faso': 12,
 'Mali': 12,
 'Ethiopia': 11,
 'South Africa': 10,
 'Sudan': 9,
 'Togo': 8,
 'Gabon': 8,
 'Angola': 8,
 'Uganda': 7,
 'Congo': 7,
 'Kenya': 6,
 'Zimbabwe': 5,
 'Benin': 4,
 'Mozambique': 4,
 'Sierra Leone': 3,
 'Malawi': 3,
 'Libya': 3,
 'Namibia': 3,
 'Equatorial Guinea': 3,
 'Cape Verde': 3,
 'Guinea-Bissau': 3,
 'Liberia': 2,
 'Niger': 2,
 'Tanzania': 2,
 'Mauritania': 2,
 'Burundi': 1,
 'Comoros': 1,
 'Mauritius': 1,
 'Rwanda': 1,
 'Madagascar': 1,
 'Botswana': 1,
 'Gambia': 1}

In [120]:
dict_team_nb_games = total_games.to_dict()
dict_team_nb_games

{'Egypt': 107,
 'Ghana': 102,
 'Ivory Coast': 99,
 'Nigeria': 96,
 'Cameroon': 91,
 'Tunisia': 79,
 'Algeria': 77,
 'DR Congo': 73,
 'Morocco': 70,
 'Zambia': 67,
 'Senegal': 67,
 'Mali': 54,
 'Burkina Faso': 48,
 'Guinea': 47,
 'South Africa': 43,
 'Ethiopia': 30,
 'Sudan': 27,
 'Congo': 26,
 'Angola': 26,
 'Gabon': 25,
 'Togo': 25,
 'Uganda': 23,
 'Kenya': 17,
 'Zimbabwe': 15,
 'Equatorial Guinea': 15,
 'Benin': 14,
 'Mozambique': 12,
 'Cape Verde': 11,
 'Libya': 11,
 'Malawi': 10,
 'Namibia': 9,
 'Guinea-Bissau': 9,
 'Sierra Leone': 8,
 'Mauritania': 6,
 'Niger': 6,
 'Tanzania': 6,
 'Liberia': 5,
 'Madagascar': 5,
 'Gambia': 5,
 'Comoros': 4,
 'Botswana': 3,
 'Rwanda': 3,
 'Mauritius': 3,
 'Burundi': 3}

In [121]:
df_editions_matches = pd.DataFrame({'nb_editions' : dict_team_nb_editions, 'nb_games' : dict_team_nb_games})
df_editions_matches

,nb_editions,nb_games
Egypt,25,107
Ivory Coast,24,99
Ghana,23,102
Tunisia,20,79
Cameroon,20,91
Nigeria,19,96
DR Congo,19,73
Algeria,19,77
Morocco,18,70
Zambia,17,67


In [122]:
df_editions_matches.reset_index(inplace=True)

In [123]:
df_editions_matches.rename(columns={"index" : "team"}, inplace=True)
df_editions_matches

,team,nb_editions,nb_games
0,Egypt,25,107
1,Ivory Coast,24,99
2,Ghana,23,102
3,Tunisia,20,79
4,Cameroon,20,91
5,Nigeria,19,96
6,DR Congo,19,73
7,Algeria,19,77
8,Morocco,18,70
9,Zambia,17,67
